In [74]:
import json
import requests
import pandas as pd

url = "https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/main/nyse/nyse_full_tickers.json"

resp = requests.get(url)
data = json.loads(resp.text)
pd.DataFrame(data[:5])

,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url
0,A,Agilent Technologies Inc. Common Stock,$137.49,-1.71,-1.228%,1553171,40292170997.00,United States,1999,Electrical Products,Industrials,/market-activity/stocks/a
1,AA,Alcoa Corporation Common Stock,$36.08,0.02,0.055%,5699843,6478488359.00,United States,2016,Aluminum,Industrials,/market-activity/stocks/aa
2,AACT,Ares Acquisition Corporation II Class A Ordina...,$10.615,0.005,0.047%,13734,0.00,,2023,Blank Checks,Finance,/market-activity/stocks/aact
3,AAN,Aarons Holdings Company Inc. Common Stock,$7.43,0.08,1.088%,222591,225585455.00,United States,2020,Diversified Commercial Services,Consumer Discretionary,/market-activity/stocks/aan
4,AAP,Advance Auto Parts Inc.,$76.53,-1.47,-1.885%,895064,4557441244.00,United States,,Auto & Home Supply Stores,Consumer Discretionary,/market-activity/stocks/aap


In [138]:
import yfinance as yf

tickers = ["AES", "AESI", "AAPL", "META"]

f = []
for ticker in tickers:
    a = ["symbol", "beta", "revenueGrowth", "priceToBook", "debtToEquity", "profitMargins", "quickRatio", 
         "fiftyDayAverage", "pegRatio"]
    try:
        f.append({ticker: dict(filter(lambda item: item[0] in a, yf.Ticker(ticker).info.items()))})
    except KeyError:
        continue
    except ValueError:
        continue


f

[{'AES': {'beta': 1.087,
   'fiftyDayAverage': 16.4654,
   'profitMargins': 0.01966,
   'priceToBook': 6.9521103,
   'pegRatio': 1.21,
   'symbol': 'AES',
   'quickRatio': 0.347,
   'debtToEquity': 379.595,
   'revenueGrowth': -0.03}},
 {'AESI': {'fiftyDayAverage': 21.1666,
   'profitMargins': 0.26059,
   'priceToBook': 2.667704,
   'pegRatio': -30.0,
   'symbol': 'AESI',
   'quickRatio': 3.039,
   'debtToEquity': 20.51,
   'revenueGrowth': -0.058}},
 {'AAPL': {'beta': 1.276,
   'fiftyDayAverage': 173.886,
   'profitMargins': 0.26163,
   'priceToBook': 35.44544,
   'pegRatio': 2.69,
   'symbol': 'AAPL',
   'quickRatio': 0.92,
   'debtToEquity': 145.803,
   'revenueGrowth': 0.021}},
 {'META': {'fiftyDayAverage': 495.4192,
   'profitMargins': 0.32062998,
   'priceToBook': 7.5123396,
   'pegRatio': 1.09,
   'symbol': 'META',
   'quickRatio': 2.546,
   'debtToEquity': 25.168,
   'revenueGrowth': 0.273}}]

In [125]:
a = ["beta", "revenueGrowth", "priceToBook", "debtToEquity", "profitMargins", "quickRatio", 
    "fiftyDayAverage", "pegRatio"]

tick = {"AAPL": dict(filter(lambda item: item[0] in a, yf.Ticker("AAPL").info.items()))}

pd.DataFrame(tick)

,AAPL
beta,1.27600
debtToEquity,145.80300
fiftyDayAverage,173.88600
pegRatio,2.69000
priceToBook,35.44544
profitMargins,0.26163
quickRatio,0.92000
revenueGrowth,0.02100
